# Plasticity with the Modified Cam-Clay model

In this section, we will introduce the implementation of plasticity which requires solving an *implicit* set of equations.

## Some recalls on plasticity

In the theory of flow plasticity, it is assumed that strain can be split into an elastic and a plastic contributions, for instance additively (or sometimes multiplicatively):
$$
\tens{\varepsilon} = \tens{\varepsilon}^e + \tens{\varepsilon}^p.
$$

The onset of plasticity is governed by the *yield function* $f$, which depends on the stress tensor $\tens{\sigma}$ and potentially on some hardening variables $\alpha$.
The domain of admissible stress is defined by:

$$
f(\tens{\sigma}, \alpha) \leq 0,
$$

where a strict inequality indicates elastic deformations, while plasticity is triggered when the stress state is situated at the edge of the domain, when $f(\tens{\sigma}, \alpha) = 0$.
The plastic strain rate can be calculated as:

$$
\tens{\dot{\varepsilon}}^p = \dot{\lambda}\frac{\partial g}{\partial \tens{\sigma}},
$$

where $\lambda$ is the so-called *plastic multiplier* and $g$ is the plastic flow rule.
In many cases, we take $g=f$ (associated plasticity).

## The Modified Cam-Clay (MCC) model

The yield function of the Modified Cam-Caly model is defined as follows:
$$
f(p,q,p_c) = q^2 + M^2p(p-p_c),
$$

where  $p=\frac{1}{3}\mathrm{tr}(\tens{\sigma})$ is the mean stress and $q = \sqrt{\frac{3}{2}\tens{\sigma}^d:\tens{\sigma}^d}$.

$$
\dot{p_c} = \frac{1+e_0}{\lambda-\kappa}\mathrm{tr}(\tens{\dot{\varepsilon}}^p)p_c
$$

## Implementation

```cpp
@DSL ImplicitGenericBehaviour;
@Author Maxime PIERRE;
@Behaviour CamClay;
@Algorithm NewtonRaphson_NumericalJacobian ;
@Theta 1. ;
@Epsilon 10e-14 ;
@PerturbationValueForNumericalJacobianComputation 1.e-9;


@Gradient StrainStensor εᵗᵒ;
εᵗᵒ.setGlossaryName("Strain");

@Flux StressStensor σ;
σ.setGlossaryName("Stress");

@MaterialProperty stress E;
E.setGlossaryName("YoungModulus");
@MaterialProperty real ν;
ν.setGlossaryName("PoissonRatio");
@MaterialProperty real b;
@MaterialProperty real k;
k.setEntryName("ACC_k");
@MaterialProperty real M;
M.setEntryName("ACC_M");
@MaterialProperty real h_ε ;
h_ε.setEntryName("VolumetricHardening");

@StateVariable StrainStensor ε_el;
ε_el.setGlossaryName("ElasticStrain");
@StateVariable real λ_p;
λ_p.setGlossaryName("EquivalentPlasticStrain");
@AuxiliaryStateVariable stress pc;
pc.setEntryName("PreconsolidationPressure");
@IntegrationVariable strain rpc ;

@AuxiliaryStateVariable stress p;
@AuxiliaryStateVariable stress q;

@LocalVariable stress λ;
@LocalVariable stress μ;
@LocalVariable real F_el;
@LocalVariable Stensor n;

@TangentOperatorBlocks{∂σ∕∂Δεᵗᵒ};
```
